In [14]:
import pandas
import boto3
from boto3.dynamodb.conditions import Key

# Get Data: 
photosynthesis whole

In [15]:
tblName = "ResourceDocuments"
nodeIdentifierName = ""

In [16]:
# Get Definitions for photosynthesis from dynamodb

## Connect to dynamodb
dynamodbClient = boto3.resource("dynamodb")

## Connect to table with resources
resourceTbl = dynamodbClient.Table(tblName)

In [17]:
response = resourceTbl.query(
    IndexName='NODE_IDENTIFIER-RESOURCE_DATATYPE-index',
    KeyConditionExpression=Key('NODE_IDENTIFIER').eq(nodeIdentifierName)
)

text_df = pd.DataFrame(response["Items"])
logger.info("Db Response Shape: {}".format(text_df.shape))

text_df.drop_duplicates(['RESOURCE'], keep='last', inplace=True)


ClientError: An error occurred (ValidationException) when calling the Query operation: ExpressionAttributeValues contains invalid value: One or more parameter values were invalid: An AttributeValue may not contain an empty string for key :v0